# Filter in Pyrosetta (API)

@Author: 黄健
@E-mail: jian.huang@xtalpi.com

@Proofread: 吴炜坤
@email：weikun.wu@xtalpi.com

更多参考:
https://new.rosettacommons.org/docs/latest/scripting_documentation/RosettaScripts/Filters/Filters-RosettaScripts

</br>

本章节将详细介绍Pyrosetta中一些常用的filter的使用，并给出示例。请读者根据自己需求，需要使用时进行查询即可。

In [1]:
from pyrosetta.rosetta.protocols.rosetta_scripts import *
from pyrosetta import *
init()

PyRosetta-4 2020 [Rosetta PyRosetta4.Release.python36.ubuntu 2020.28+release.8ecab77aa50ac1301efe53641e07e09ac91fee3b 2020-07-07T16:41:06] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.Release.python36.ubuntu r260 2020.28+release.8ecab77aa50 8ecab77aa50ac1301efe53641e07e09ac91fee3b http://www.pyrosetta.org 2020-07-07T16:41:06
core.init: command: PyRosetta -ex1 -ex2aro -database /home/huangjian/miniconda3/envs/biodesign/lib/python3.6/site-packages/pyrosetta-2020.28+release.8ecab77aa50-py3.6-linux-x86_64.egg/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=-545887235 seed_offset=0 real_seed=-545887235
basic.random.init_random_generator: RandomGenerator:init: Normal mode, seed=-545887235 RG_type=mt19937


In [2]:
# 定义一个模板函数，让我们可以很方便创建各种filter的RS
def generate_filter_RS(residue_selector_str:str="", simple_metric_str:str="", filter_definition_str:str="", protocol_definition_str:str=""):
    filter_template = \
    """
    <ROSETTASCRIPTS>
            <SCOREFXNS>
            </SCOREFXNS>
            <RESIDUE_SELECTORS>
                    {}
            </RESIDUE_SELECTORS>
            <TASKOPERATIONS>
            </TASKOPERATIONS>
            <SIMPLE_METRICS>
                    {}
            </SIMPLE_METRICS>
            <FILTERS>
                    {}
            </FILTERS>
            <MOVERS>
            </MOVERS>
            <APPLY_TO_POSE>
            </APPLY_TO_POSE>
            <PROTOCOLS>
                    {}
            </PROTOCOLS>
            <OUTPUT/>
    </ROSETTASCRIPTS>
    """.format(residue_selector_str, simple_metric_str, filter_definition_str, protocol_definition_str)
    return filter_template

## 1. SimpleMetricFilter -- 衍生于SimpleMetric的Filter

```
<SimpleMetricFilter name="(&string;)" epsilon="(.0001 &real;)"
        metric="(&string;)" comparison_type="(&comparison_types;)"
        composite_action="(&string;)" cutoff="(&real;)" match="(&string;)"
        use_sum_for_per_residue_real="(&bool;)" use_cached_data="(false &bool;)"
        cache_prefix="(&string;)" cache_suffix="(&string;)"
        fail_on_missing_cache="(true &bool;)" confidence="(1.0 &real;)" >
    <Simple Metric Tag ... />
</SimpleMetricFilter>
```

1. epsilon: epsilon for numerical comparisons
2. metric:在SIMPLE_METRICS中定义的SimpleMetric
3. comparison_type：比较类型，可选 “nq”：not equal；“eq”： equal etc, choices are [eq, gt, gt_or_eq, lt, lt_or_eq, ne]
4. composite_action:Composite Metrics的一个额外选项，可选：any，all，metric_name
5. cutoff： filter用以判定是否通过的数值
6. match： filter用以判定是否通过的string类型，针对StringMetric和CompositeStringMetric
7. use_sum_for_per_residue_real：在使用PerResidueRealMetric可设置为Ture，使用总体的值作为判定
8. use_cached_data：使用缓存数据。Metric的数据缓存在SimpleMetric的Mover-RunSimpleMetrics运行之后。
9. cache_prefix： 若use_cached_data为True，则指明RunSimpleMetrics的前缀
10. cache_suffix： 若use_cached_data为True，则指明RunSimpleMetrics的后缀
11. fail_on_missing_cache：若use_cached_data为True但有没有找到缓存数据，是否报告失败
12. confidence：若一个pose没有通过filter，有多大几率被过滤掉。Rosetta中所有Filter的默认值均为1。

In [22]:
# 使用layer选择器选取表面层的残基
residue_selector_definition_1 = '<Layer name="surfacelayer" select_core="false" select_boundary="false" select_surface="true" core_cutoff="4.0" />'

# 注意simpleMetric定义中指定selector对象名字与选择器中保持一致
simple_metric_definition_1 = '<SasaMetric name="sasa" residue_selector="surfacelayer"/>'

# 定义filter层：这里展示了溶剂可表面的metric的filter的例子。其他的simpleMetric请参考相关章节
simple_metric_filter_definition_1 = '<SimpleMetricFilter name="sasa_filter" metric="sasa" cutoff="100" comparison_type="gt"/>'

# 在protocol层添加该filter
protocol_definition_1 = '<Add filter_name="sasa_filter" />'

simpleMetricFilter_example = generate_filter_RS(residue_selector_definition_1,simple_metric_definition_1,simple_metric_filter_definition_1,  protocol_definition_1)

print(simpleMetricFilter_example)


    <ROSETTASCRIPTS>
            <SCOREFXNS>
            </SCOREFXNS>
            <RESIDUE_SELECTORS>
                    <Layer name="surfacelayer" select_core="false" select_boundary="false" select_surface="true" core_cutoff="4.0" />
            </RESIDUE_SELECTORS>
            <TASKOPERATIONS>
            </TASKOPERATIONS>
            <SIMPLE_METRICS>
                    <SasaMetric name="sasa" residue_selector="surfacelayer"/>
            </SIMPLE_METRICS>
            <FILTERS>
                    <SimpleMetricFilter name="sasa_filter" metric="sasa" cutoff="100" comparison_type="gt"/>
            </FILTERS>
            <MOVERS>
            </MOVERS>
            <APPLY_TO_POSE>
            </APPLY_TO_POSE>
            <PROTOCOLS>
                    <Add filter_name="sasa_filter" />
            </PROTOCOLS>
            <OUTPUT/>
    </ROSETTASCRIPTS>
    


In [23]:
xml_simpleMetricFilter = XmlObjects.create_from_string(simpleMetricFilter_example)

protocols.rosetta_scripts.RosettaScriptsParser: Generating XML Schema for rosetta_scripts...
protocols.rosetta_scripts.RosettaScriptsParser: ...done
protocols.rosetta_scripts.RosettaScriptsParser: Initializing schema validator...
protocols.rosetta_scripts.RosettaScriptsParser: ...done
protocols.rosetta_scripts.RosettaScriptsParser: Validating input script...
protocols.rosetta_scripts.RosettaScriptsParser: ...done
protocols.rosetta_scripts.RosettaScriptsParser: Parsed script:
<ROSETTASCRIPTS>
	<SCOREFXNS/>
	<RESIDUE_SELECTORS>
		<Layer core_cutoff="4.0" name="surfacelayer" select_boundary="false" select_core="false" select_surface="true"/>
	</RESIDUE_SELECTORS>
	<TASKOPERATIONS/>
	<SIMPLE_METRICS>
		<SasaMetric name="sasa" residue_selector="surfacelayer"/>
	</SIMPLE_METRICS>
	<FILTERS>
		<SimpleMetricFilter comparison_type="gt" cutoff="100" metric="sasa" name="sasa_filter"/>
	</FILTERS>
	<MOVERS/>
	<APPLY_TO_POSE/>
	<PROTOCOLS>
		<Add filter_name="sasa_filter"/>
	</PROTOCOLS>
	<OUTPUT/>

In [25]:
simpleMetricFilter = xml_simpleMetricFilter.get_filter("sasa_filter")

# apply to pose
# simpleMetricFilter.apply(pose)
# simpleMetricFilter.score(pose)

## 2. 基本的Filter类型 -- ResidueCount和NetCharge

ResidueCount：基于残基总数的过滤器，其官方完整定义如下：

```
<ResidueCount name="(&string)" max_residue_count="(Inf &Integer)" min_residue_count="(0 &Integer)" residue_types="('' &string)" count_as_percentage="(0 &Bool)" residue_selector="selector" />
```
1. residue_types: 逗号分隔的残基类型名字的列表，如"CYS,SER,HIS_D"，只有在列表中的残基会被记数
2. include_property（未展示）：逗号分隔的性质列表，如"HYDROPHOBIC,ALIPHATIC"，只有在列表中的性质的残基会被记数
3. max_residue_count：计数残基的总数最大允许值
4. min_residue_count：计数残基的总数最小允许值
5. count_as_percentage：设定为True表示使用百分数占比(=100*raw_number_of_specified_residue/total_residue)表示，而非计数总数
6. residue_selector：残基选择器，选定计数范围
7. task_operations：指定计数残基的子集，默认仅计数可设计的残基
8. packable：T/F，计数在task_operations中允许repack的残基

NetCharge：基于蛋白总电荷值的过滤器，其官方完整定义如下：

```
<NetCharge name="(&string)" min="(-100 &Integer)" max="(100 &Integer)" chain="(0 &Integer)" task_operations="('' &string)" />
```
1. min：最少净电荷（默认-100）
2. max：最大净电荷（默认100）
3. chain：指定那一条链作为电荷计数的链（从PDB文件头到尾，依次为1，2，3...）默认为0，计数所有chain的电荷
4. task_operations：在task_operations指定可设计的残基的电荷计数，不可设计的残基不再计数

NetCharge设定LYS和ARG残基电荷值为+1，酸性残基ASP和GLU电荷值为-1。

**注意若不显式设置task_operations，rosetta会默认所有残基均可设计**


In [31]:
# 使用layer选择器选取表面层的残基
residue_selector_definition = '<Layer name="surfacelayer" select_core="false" select_boundary="false" select_surface="true" core_cutoff="4.0" />'

# 定义filter层
residue_count_filter_definition = '<ResidueCount name="residue_count" max_residue_count="10" min_residue_count="5" \
residue_types = "CYS" residue_selector="surfacelayer"/>'

# 在protocol层添加该filter
residue_count_protocol_definition = '<Add filter_name="residue_count" />'

residue_count_Filter = generate_filter_RS(residue_selector_definition_1,"",residue_count_filter_definition,  residue_count_protocol_definition)

print(residue_count_Filter)


    <ROSETTASCRIPTS>
            <SCOREFXNS>
            </SCOREFXNS>
            <RESIDUE_SELECTORS>
                    <Layer name="surfacelayer" select_core="false" select_boundary="false" select_surface="true" core_cutoff="4.0" />
            </RESIDUE_SELECTORS>
            <TASKOPERATIONS>
            </TASKOPERATIONS>
            <SIMPLE_METRICS>
                    
            </SIMPLE_METRICS>
            <FILTERS>
                    <ResidueCount name="residue_count" max_residue_count="10" min_residue_count="5" residue_types = "CYS" residue_selector="surfacelayer"/>
            </FILTERS>
            <MOVERS>
            </MOVERS>
            <APPLY_TO_POSE>
            </APPLY_TO_POSE>
            <PROTOCOLS>
                    <Add filter_name="residue_count" />
            </PROTOCOLS>
            <OUTPUT/>
    </ROSETTASCRIPTS>
    


In [32]:
residue_count_xml = XmlObjects.create_from_string(residue_count_Filter)
residue_count_filter = xml_simpleMetricFilter.get_filter("sasa_filter")

# apply to pose
# residue_count_filter.apply(pose)
# residue_count_filter.score(pose)

protocols.rosetta_scripts.RosettaScriptsParser: Generating XML Schema for rosetta_scripts...
protocols.rosetta_scripts.RosettaScriptsParser: ...done
protocols.rosetta_scripts.RosettaScriptsParser: Initializing schema validator...
protocols.rosetta_scripts.RosettaScriptsParser: ...done
protocols.rosetta_scripts.RosettaScriptsParser: Validating input script...
protocols.rosetta_scripts.RosettaScriptsParser: ...done
protocols.rosetta_scripts.RosettaScriptsParser: Parsed script:
<ROSETTASCRIPTS>
	<SCOREFXNS/>
	<RESIDUE_SELECTORS>
		<Layer core_cutoff="4.0" name="surfacelayer" select_boundary="false" select_core="false" select_surface="true"/>
	</RESIDUE_SELECTORS>
	<TASKOPERATIONS/>
	<SIMPLE_METRICS/>
	<FILTERS>
		<ResidueCount max_residue_count="10" min_residue_count="5" name="residue_count" residue_selector="surfacelayer" residue_types="CYS"/>
	</FILTERS>
	<MOVERS/>
	<APPLY_TO_POSE/>
	<PROTOCOLS>
		<Add filter_name="residue_count"/>
	</PROTOCOLS>
	<OUTPUT/>
</ROSETTASCRIPTS>
core.scoring

In [33]:
# 使用layer选择器选取表面层的残基
residue_selector_definition = '<Layer name="surfacelayer" select_core="false" select_boundary="false" select_surface="true" core_cutoff="4.0" />'

# 定义filter层
netcharge_filter_definition = '<NetCharge name="netcharge" min="0" max="5" />'

# 在protocol层添加该filter
netcharge_protocol_definition = '<Add filter_name="netcharge" />'

netcharge_Filter = generate_filter_RS(residue_selector_definition,"",netcharge_filter_definition,  netcharge_protocol_definition)

# print(netcharge_Filter)

In [35]:
netcharge_xml = XmlObjects.create_from_string(netcharge_Filter)
netcharge_filter = netcharge_xml.get_filter("netcharge")

# apply to pose
# residue_count_filter.apply(pose)
# residue_count_filter.score(pose)

protocols.rosetta_scripts.RosettaScriptsParser: Generating XML Schema for rosetta_scripts...
protocols.rosetta_scripts.RosettaScriptsParser: ...done
protocols.rosetta_scripts.RosettaScriptsParser: Initializing schema validator...
protocols.rosetta_scripts.RosettaScriptsParser: ...done
protocols.rosetta_scripts.RosettaScriptsParser: Validating input script...
protocols.rosetta_scripts.RosettaScriptsParser: ...done
protocols.rosetta_scripts.RosettaScriptsParser: Parsed script:
<ROSETTASCRIPTS>
	<SCOREFXNS/>
	<RESIDUE_SELECTORS>
		<Layer core_cutoff="4.0" name="surfacelayer" select_boundary="false" select_core="false" select_surface="true"/>
	</RESIDUE_SELECTORS>
	<TASKOPERATIONS/>
	<SIMPLE_METRICS/>
	<FILTERS>
		<NetCharge max="5" min="0" name="netcharge"/>
	</FILTERS>
	<MOVERS/>
	<APPLY_TO_POSE/>
	<PROTOCOLS>
		<Add filter_name="netcharge"/>
	</PROTOCOLS>
	<OUTPUT/>
</ROSETTASCRIPTS>
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.select.residue_selector.LayerSelector: Se

## 3. 与能量项或打分相关的Filter

这一类的Filter包括：

**ScoreType**：基于某特定打分项的Filter
```
<ScoreType name="(&string;)" score_type="(total_score &string;)"
        threshold="(&real;)" scorefxn="(&string;)" confidence="(1.0 &real;)" />
```
1. score_type：某打分项。若缺省，则以总得分作为filter
2. threshold：必要参数，若能量值低于该值，返回True
3. scorefxn：指定打分函数的名字（在SCOREFXNS标签下应先定义）

**TaskAwareScoreType**：对可以pack的残基部分的打分的Filter
```
<TaskAwareScoreType name="(&string)" task_operations="(&comma-delimited list of taskoperations)" scorefxn="(score12 &string)" score_type="(total_score &string)" threshold="(100000 &Real)" mode="(total &string)", unbound="(0 &bool)", sym_dof_names="(&comma-delimited list of strings)", jump="(0 &Size)", write2pdb="(0 &bool)", bb_bb="(0 &bool)" />
```
1. task_operations：设定repack残基的task operations
2. scorefxn：指定打分函数的名字（在SCOREFXNS标签下应先定义），默认为score12
3. score_type：打分项
4. threshold：必要参数，若能量值低于该值，返回True。若设定mode为“individual”，则每一个残基都需要通过该阈值
5. mode：可选"total", "average", or "individual"

**BindingStrain**：在结合态的单体的能量张力的Filter
```
<BindingStrain name="(&string)" threshold="(3.0 &Real)" task_operations="(comma-delimited list of operations &string)" scorefxn="(score12 &string)" relax_mover="(null &string)" jump="(1 &Int)"/>
```
1. threshold：必要参数，允许多大能量张力
2. task_operations：设定repack残基的task operations
3. scorefxn：用来repack的打分函数
4. relax_mover：在unbound state 进行repack之后，下面需要使用何种mover进行relax（MinMover）
**ConstraintScore**：从ConstraintGenerators产生的一系列constraints计算的打分项的Filter
```
<ConstraintScore name="(&string)" constraint_generator="('', &string)" threshold="(0 &Real)" />
```
1. constraint_generators：constraint generators的名字，需要使用AddConstraints进行事先定义
2. threshold：必要参数，允许多大能量值
**Delta**：计算filter中的值与input结构能量差值
```
<Delta name="(&string)" upper="(1 &bool)" lower="(0 &bool)" range="(0 &Real)" filter="(&string)" unbound="(0 &bool)" jump="(see below &Int)" relax_mover="(null &string)"/>
```
1. upper/lower：设定阈值为最低阈值还是最高阈值
2. range：允许高出或低于基线多大范围
3. filter：提前定义的filter的名字，用以评估能量项
4. unbound：平移其他构象partner10000A，再进行评估
5. jump：与unbound结合起来使用，指定平移对象
6. relax_mover: 在设定baseline之前先进行的relax操作，尽量使能量更低
**EnergyPerResidue**：基于某特定残基选择部分的能量项的Filter
```
<EnergyPerResidue name="(energy_per_res_filter &string)" scorefxn="(score12 &string)" 
score_type="(total_score &string)" pdb_num/res_num="(&string)" energy_cutoff="(0.0 &float)"
whole_interface="(0 &bool)" jump_number="(1 &int)" interface_distance_cutoff="(8.0 &float)" bb_bb="(0, bool)" resn resns="('1' &string)" whole_protein="(0 andint)" resnums="(andstring)/>"
```
1. pdb_num/res_num：基于PDB或Rosetta编号的选取
**ReadPoseExtraScoreFilter**：提取早前储存在pose中的打分项，可以用来提取FilterReportAsPoseExtraScoresMover保存的打分记录
```
<ReadPoseExtraScoreFilter name="read_sew_total" term_name="sew_total" threshold="-1"/>
```
1. term_name：需要与可以用来提取FilterReportAsPoseExtraScoresMover中的report_as选项定义保持一致
**Residue Interaction Energy**：在界面上的特定残基的相互作用打分。计算特定残基在界面（interface=1）或整体pose（interface=0）的相互作用能量。而后如果相互作用得分高于cutoff值，给予惩罚penalty = sum (IE(res)-(cutoff))。
```
<ResidueIE name="(&string)" scorefxn="(score12 &string)" score_type="(total_score &string)" energy_cutoff="(0.0 &float)" restype3="(TRP &string; 3 letter code)" interface="(0 &bool)" whole_pose(0 &bool) selector="(&residue_selector)" jump_number="(1 &float)" interface_distance_cutoff="(8.0 &float)" max_penalty="(1000.0 &float)" penalty_factor="(1.0 & float)"/>
```
**ResidueSetChainEnergy**:残基集合的能量filter（可以是链或者某selection）
```
<ResidueSetChainEnergy name="(residue_set_chain_energy_filter &string)" scorefxn="(score12 &string)" score_type="(total_score &string)" resnums="(&string)" chain="(0 &int)" threshold="(&float)"/>
```
计算两组残基集合之间的相互作用能量。Group1使用resnum定义（兼容PDB编号和Rosetta编号），Group2由chain定义，使用rosetta chain编号定义。
**ScorePoseSegmentFromResidueSelectorFilter**：通过残基选择器（ResidueSelector）选取的部分的得分。
```
<ScorePoseSegmentFromResidueSelectorFilter name="(&string;)"
        in_context="(0 &bool;)" residue_selector="(&string;)"
        scorefxn="(&string;)" confidence="(1.0 &real;)" />
```
1. in_context:在打分前不单独区分selection
2. residue_selector:必要选项，指定事先定义的selector的名字，也可以使用逻辑符AND, NOT, OR。

In [39]:
# 此处仅以第一个ScoreType为例

# 使用layer选择器选取表面层的残基
residue_selector_definition = '<Layer name="surfacelayer" select_core="false" select_boundary="false" select_surface="true" core_cutoff="4.0" />'

# 定义filter层，这里没有指定scorefnx，将使用默认ref2015打分函数
scoretype_filter_definition = '<ScoreType name="scoretype" score_type="fa_atr" threshold = "0.5" />'

# 在protocol层添加该filter
scoretype_protocol_definition = '<Add filter_name="scoretype" />'

scoretype_Filter = generate_filter_RS(residue_selector_definition,"",scoretype_filter_definition,  scoretype_protocol_definition)

# print(scoretype_Filter)

In [40]:
scoretype_xml = XmlObjects.create_from_string(scoretype_Filter)
scoretype_filter = scoretype_xml.get_filter("scoretype")

# apply to pose
# residue_count_filter.apply(pose)
# residue_count_filter.score(pose)

protocols.rosetta_scripts.RosettaScriptsParser: Generating XML Schema for rosetta_scripts...
protocols.rosetta_scripts.RosettaScriptsParser: ...done
protocols.rosetta_scripts.RosettaScriptsParser: Initializing schema validator...
protocols.rosetta_scripts.RosettaScriptsParser: ...done
protocols.rosetta_scripts.RosettaScriptsParser: Validating input script...
protocols.rosetta_scripts.RosettaScriptsParser: ...done
protocols.rosetta_scripts.RosettaScriptsParser: Parsed script:
<ROSETTASCRIPTS>
	<SCOREFXNS/>
	<RESIDUE_SELECTORS>
		<Layer core_cutoff="4.0" name="surfacelayer" select_boundary="false" select_core="false" select_surface="true"/>
	</RESIDUE_SELECTORS>
	<TASKOPERATIONS/>
	<SIMPLE_METRICS/>
	<FILTERS>
		<ScoreType name="scoretype" score_type="fa_atr" threshold="0.5"/>
	</FILTERS>
	<MOVERS/>
	<APPLY_TO_POSE/>
	<PROTOCOLS>
		<Add filter_name="scoretype"/>
	</PROTOCOLS>
	<OUTPUT/>
</ROSETTASCRIPTS>
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.select.residue_select

## 4. 距离相关的Filter

在第一节中我们学习了AtomicDistance filter进行两个原子之间距离的计算和设置过滤条件。关于距离的Filter还有以下几种：

**ResidueDistance**：残基距离，以每个残疾的邻原子作为计算（通常为C-β原子）
```
<ResidueDistance name="(&string)" res1_res_num="(&string)" res1_pdb_num="(&string)" res2_res_num="(&string)" res2_pdb_num="(&string)" distance="(8.0 &Real)"/>
```
1. 这里可以使用rosetta编号和pdb编号，选择其一即可。
**AtomicContact**：两个残基是否存在任何一对原子距离在设定的cutoff内？
```
<AtomicContact name="(&string)" residue1="(&integer)" residue2="(&integer)" sidechain="1" backbone="0" protons="0" distance="(4.0 &integer)"/>
```
1. 用户可以指定sidechain，backbone，protons指定计算距离对的范围
**AtomicContactCount**：在指定残基之间计算侧链C原子与C原子的距离是否在设定cutoff内
```
"All" mode:
<AtomicContactCount name="(&string)" partition="all" task_operations="(comma-delimited list of operations &string)"  distance="(4.5 &integer)"/>

"jump" mode
<AtomicContactCount name="(&string)" partition="jump" task_operations="(comma-delimited list of operations &string)"  distance="(4.5 &integer)" jump="(1 &integer)" normalize_by_sasa="(0 &bool)"/>

"chain" mode
<AtomicContactCount name="(&string)" partition="chain" task_operations="(comma-delimited list of operations &string)"  distance="(4.5 &integer)" normalize_by_sasa="(0*|detect_by_task|1 &str)"/>
```
在给定task operations定义可以pack的残基中依次迭代计算所有侧链间C原子间的距离（若没有指定task operations，默认使用所有残基）。可以被限制为计算jump间的contact或chain之间的contact（jump mode或chain mode）
**TerminusDistance**：如果所有表面残基离N或C端距离长于n个残基，即pass/True，返回1000。False，则返回距离短于n个残基的残基距离末端的距离。
```
<TerminusDistance name="(&string)" jump_number="(1 &integer)" distance="(5 &integer)"/>
```

*关于距离的示例请参考本章第一节内容，在此不再赘述*

## 5. 基于序列分析的Filter

**LongestContinuousPolarSegment**: 基于极性氨基酸残基最大长度的Filter
```
<LongestContinuousPolarSegment name=(string)
     exclude_chain_termini=(bool,"true")
     count_gly_as_polar=(bool,"true")
     filter_out_high=(bool,"true") cutoff=(int,"5")
     residue_selector=(string) confidence=(real,"1.0")
/>
```
1. exclude_chain_termini： false表示极性区域能够延展到N端或C端的将被计算；true表示不被计算（默认为true，仅内部的极性残基块被计算）
2. count_gly_as_polar： true表示gly会被考虑为极性氨基酸，（默认为true）
3. filter_out_high ：true表示高于cutoff设定值的极性残基长度的pose会被reject掉；false表示低于cutoff会被reject（默认为true）
4. cutoff：最长极性残基长度的阈值，默认值为5
5. residue_selector：氨基酸选择器，应预先定义
**LongestContinuousApolarSegment**：基于非极性氨基酸残基最大长度的Filter。*rosetta中默认定义ACFGILMPVWY为极性，其余为非极性残基*。
默认该filter将reject掉那些含有最大非极性残基长度超过设定cutoff值的pose。
```
<LongestContinuousApolarSegment name=(string)
     exclude_chain_termini=(bool,"true")
     filter_out_high=(bool,"true") cutoff=(int,"5")
     residue_selector=(string) confidence=(real,"1.0")
/>
```
选项跟上一个基本类似，不再赘述。
**SequenceDistanceFilter**：基于pose序列和pose的comment中储存序列的突变的数目的Filter。计算当前pose序列与pose comments中储存的一个序列或是使用target_sequence指定的序列之间的hamming距离
```
< SequenceDistance name=(string)
     sequence_comment_id=(string,"true")
     target_sequence=(string,"true")
     threshold=(int,"8000") 
     confidence=(real,"1.0")
/>
```
1. sequence_comment_id： comment id， 需要从pose的comment该id中载入序列。通常会先用SaveSequenceToComments保存一个comment sequence
2. target_sequence：用于与pose序列对比的氨基酸序列
3. threshold： 最大hamming distance，默认值为8000

In [3]:
def generate_filter_RS_2(residue_selector_str:str="", simple_metric_str:str="", filter_definition_str:str="", mover_definition_str:str="", protocol_definition_str:str=""):
    filter_template = \
    """
    <ROSETTASCRIPTS>
            <SCOREFXNS>
            </SCOREFXNS>
            <RESIDUE_SELECTORS>
                    {}
            </RESIDUE_SELECTORS>
            <TASKOPERATIONS>
            </TASKOPERATIONS>
            <SIMPLE_METRICS>
                    {}
            </SIMPLE_METRICS>
            <FILTERS>
                    {}
            </FILTERS>
            <MOVERS>
                    {}
            </MOVERS>
            <APPLY_TO_POSE>
            </APPLY_TO_POSE>
            <PROTOCOLS>
                    {}
            </PROTOCOLS>
            <OUTPUT/>
    </ROSETTASCRIPTS>
    """.format(residue_selector_str, simple_metric_str, filter_definition_str, mover_definition_str, protocol_definition_str)
    return filter_template

In [19]:
# 写入一个允许design的Packing Mover
sequence_distance_mover_definition = '<SaveSequenceToCommentsMover name="save_sequence_to_comment" save_seq_name="seq_1" /> \
                                                                        <PackRotamersMover name="pack1" />'

# 定义filter层，这里没有指定scorefnx，将使用默认ref2015打分函数
sequence_distance_filter_definition = '<SequenceDistance  name="sequence_distance" sequence_comment_id ="seq_1" threshold = "8000" />'

# 在protocol层添加该filter
sequence_distance_protocol_definition = '<Add mover="save_sequence_to_comment" /> \
                                                    <Add mover="pack1" /> \
                                                    <Add filter_name="sequence_distance" />'

sequence_distance_Filter = generate_filter_RS_2("","",sequence_distance_filter_definition, sequence_distance_mover_definition,  sequence_distance_protocol_definition)

print(sequence_distance_Filter)


    <ROSETTASCRIPTS>
            <SCOREFXNS>
            </SCOREFXNS>
            <RESIDUE_SELECTORS>
                    
            </RESIDUE_SELECTORS>
            <TASKOPERATIONS>
            </TASKOPERATIONS>
            <SIMPLE_METRICS>
                    
            </SIMPLE_METRICS>
            <FILTERS>
                    <SequenceDistance  name="sequence_distance" sequence_comment_id ="seq_1" threshold = "8000" />
            </FILTERS>
            <MOVERS>
                    <SaveSequenceToCommentsMover name="save_sequence_to_comment" save_seq_name="seq_1" />                                                                         <PackRotamersMover name="pack1" />
            </MOVERS>
            <APPLY_TO_POSE>
            </APPLY_TO_POSE>
            <PROTOCOLS>
                    <Add mover="save_sequence_to_comment" />                                                     <Add mover="pack1" />                                                     <Add filter_name="se

In [13]:
original_pose = pose_from_pdb("./data/1ubq_clean.pdb")

pose = original_pose.clone()

core.import_pose.import_pose: File './data/1ubq_clean.pdb' automatically determined to be of type PDB


In [20]:
sequence_distance_xml = XmlObjects.create_from_string(sequence_distance_Filter)
sequence_distance_filter = sequence_distance_xml.get_filter("sequence_distance")

protocols.rosetta_scripts.RosettaScriptsParser: Generating XML Schema for rosetta_scripts...
protocols.rosetta_scripts.RosettaScriptsParser: ...done
protocols.rosetta_scripts.RosettaScriptsParser: Initializing schema validator...
protocols.rosetta_scripts.RosettaScriptsParser: ...done
protocols.rosetta_scripts.RosettaScriptsParser: Validating input script...
protocols.rosetta_scripts.RosettaScriptsParser: ...done
protocols.rosetta_scripts.RosettaScriptsParser: Parsed script:
<ROSETTASCRIPTS>
	<SCOREFXNS/>
	<RESIDUE_SELECTORS/>
	<TASKOPERATIONS/>
	<SIMPLE_METRICS/>
	<FILTERS>
		<SequenceDistance name="sequence_distance" sequence_comment_id="seq_1" threshold="8000"/>
	</FILTERS>
	<MOVERS>
		<SaveSequenceToCommentsMover name="save_sequence_to_comment" save_seq_name="seq_1"/>
		<PackRotamersMover name="pack1"/>
	</MOVERS>
	<APPLY_TO_POSE/>
	<PROTOCOLS>
		<Add mover="save_sequence_to_comment"/>
		<Add mover="pack1"/>
		<Add filter_name="sequence_distance"/>
	</PROTOCOLS>
	<OUTPUT/>
</ROSETT

In [23]:
# 按照protocol依次apply
save_sequence_filter = sequence_distance_xml.get_mover("save_sequence_to_comment")
save_sequence_filter.apply(pose)

# this step is going to take a long time to finish
packer = sequence_distance_xml.get_mover("pack1")
packer.apply(pose)

sequence_distance_filter.apply(pose)

basic.io.database: Database file opened: scoring/score_functions/elec_cp_reps.dat
core.scoring.elec.util: Read 40 countpair representative atoms
core.pack.dunbrack.RotamerLibrary: shapovalov_lib_fixes_enable option is true.
core.pack.dunbrack.RotamerLibrary: shapovalov_lib::shap_dun10_smooth_level of 1( aka lowest_smooth ) got activated.
core.pack.dunbrack.RotamerLibrary: Binary rotamer library selected: /home/huangjian/miniconda3/envs/biodesign/lib/python3.6/site-packages/pyrosetta-2020.28+release.8ecab77aa50-py3.6-linux-x86_64.egg/pyrosetta/database/rotamer/shapovalov/StpDwn_0-0-0/Dunbrack10.lib.bin
core.pack.dunbrack.RotamerLibrary: Using Dunbrack library binary file '/home/huangjian/miniconda3/envs/biodesign/lib/python3.6/site-packages/pyrosetta-2020.28+release.8ecab77aa50-py3.6-linux-x86_64.egg/pyrosetta/database/rotamer/shapovalov/StpDwn_0-0-0/Dunbrack10.lib.bin'.
core.pack.dunbrack.RotamerLibrary: Dunbrack 2010 library took 0.234375 seconds to load from binary
core.pack.pack_rot

False

## 6. 基于Alignment分析的Filter

**AlignmentAAFinder**：对一个alignment进行扫描，在每个位点测试所有可能的氨基酸，生成通过的氨基酸的文件。
扫描一个pose序列的所有位点，并测试那些同源序列中的氨基酸替代是否能够适应到当前pose中
```
< AlignmentAAFinder name="(&string)" 
    scorefxn="(beta_nov16 &string)" 
    exclude_AA_threshold="(10.0 &float)" 
    alignment_file="(null &string)" 
    loop_seqid_threshold="(0.50 &float)" 
    indel_motif_radius="(2 &int)" 
    available_AAs_file="(null &string)"
/>
```
1. exclude_AA_threshold：用于决定一个同源氨基酸位点替代后是否fit的能量阈值
2. loop_seqid_threshold：loop区域的氨基酸相似度的阈值
3. indel_motif_radius： indel motif半径
4. available_AAs_file：输出文件路径
**AlignmentGapInserter**：扫描一个alignment，在那些不能代表pose序列化学环境的位点插入gap
一般而言，读取由AlignmentAAFinder产生的available_AAs_file，其中包含了所有能够fit到pose序列的各个位点氨基酸信息。依次迭代所有的align的序列，联合到pose的序列中（忽略插入并用gly替代gap），判断化学环境是否变化。
```
< AlignmentGapInserterFilter name="(&string)" 
    scorefxn="(beta_nov16 &string)" 
    nbr_e_threshold="(0.1 &float)" 
    indel_motif_radius="(2 &int)" 
    alignment_file="(null &string)" 
    available_AAs_file="(null &string)"
    cleaned_alignment_file="(null &string)"
    loop_seqid_threshold="(comma separated floats)" 
    max_score_diffs="(comma separated floats)"
    only_clean_seq_num="(null &int)"
/>
```
1. nbr_e_threshold： 用于定义临近残基的绝对能量值（res-res-energy）
2. cleaned_alignment_file： output alignment

## 7. Geometry相关Filter

**AngleToVector**：计算一条链第一个氨基酸的两个原子或一个预先定义矢量的原子间的角度。其目的是测试一个小分子是否以正确角度结合在蛋白的口袋中。
```
<AngleToVector name="(&string)" atm1="(&string)" atm2="(&string)" chain="(2&size)" refx="(&real)" refy="(&real)" refz="(&real)" min_angle="(0.0&real)" max_angle="(90.0&real)"/>
```
1. atm1 atm2 为用于计算方向矢量的两个原子，例如 “CA” “NE”等
2. chain：ligand所在的链
3. refx, refy, refz：参照矢量的x，y，z坐标，该矢量在计算角度之前会先被归一化
其过程是：先计算两个原子之间的是矢量，然后计算这个矢量与参照矢量之间的角度

**Torsion**：基于二面角的Filter
```
<Torsion name="(&string)" lower="(0&Real)" upper="(0&Real)" resnum="(0&residue number)" torsion="('' &string)" task_operations="(&comma-delimited list of taskoperations)"/>
```
1. lower和upper:最低阈值、最高阈值
2. resnum：pdb或rosetta numbering
3. torsion：phi、psi、“”
4. task_operations：输出的residue可以是在task_operations定义过的那些残基，所有可以design的残基将被输出。不能同时使用task_operations和resnnum两个选项！
不设置torsion选项将会输出所有phi和psi；不设置resnum会report所有残基。

**HelixPairing**：基于helix成对的几何性质的Filter
该filter首先计算一个pose的二级结构，然后利用二级结构去找到螺旋部分。对于螺旋pairing，该filter提供三个参数，dist，cross和align，一个pose的这些参数低于设定阈值的话将被reject掉。
```
<HelixPairing name="(&string)"
              secstruct="('' &string)"
              use_dssp="(false &bool)"
              blueprint="('' &string)"
              helix_pairings="('' &string)"
              dist="(15.0 &Real)"
              cross="(45.0 &Real)"
              align="(25.0 &Real)"
              bend="(20.0 &Real)"
              output_id="(1 &Integer)"
              output_type="('dist' &string)" />
```

1. secstruct：使用给定的二级结构定义强制到当前pose，且用于搜索helices。二级结构长度必须与pose序列一致
2. use_dssp：使用dssp算法计算二级结构
3. helix_pairings：若给定，则给定的成对helics将作为filter条件。helic number用数字表示，两个螺旋间“-”连接，A和P表示平行方向，如 1-2.A;2-3.A;1-3.P
4. dist：成对螺旋的各自中点间的距离
5. cross：helix矢量之间的角度。helix矢量即螺旋的C端和N端中心形成的矢量
6. align： helix矢量投影到该helices之前的strand形成的beta sheet上之间的角度
7. bend：检查螺旋内部bend的程度
8. output_id：输出到score file中的成对helix的id
9. output_type：输出到score file中的参数类型：dist, cross, or align
10. blueprint：用于设定所需pose二级结构的蓝图文件

**SSMotifFinder**：基于loop主干骨架几何性质的filter。计算主干骨架几何性质与模板loop主干的相似度。该filter会在一级序列一一定长度氨基酸分隔的两个残基之间搜索，以一定RMSD cutoff值重叠模板loop的骨架原子。RMSD计算会包括主干区骨架重原子以及前后6个残基。输出为一个包含所有成对主干的pairs和特定RMSD cutoff，并给出RMSD。
```
<SSMotifFinder name="(&string)" template_pose="(&string)" template_stem1="(0 &string)" template_stem2="(0 &string)" from_res="(0 &non_negative_integer)" to_res="(0 &non_negative_integer)" rmsd="(0.0 &real)" filename="(&string)" pdbname="(&string)"/>
```
**SecondaryStructure**：基于二级结构的Filter。比较pose的二级结构与理想定义二级结构。报告N_MATCHING / N_TOTAL的值，N_MATCHING 为选取部分的残基中有多少与定义理想的二级结构一致，N_TOTAL 为选取的残基总数。若compute_pose_secstruct_by_dssp 选项为false（默认为false），必须实现为pose计算二级结构信息，例如使用DsspMover；设定为true则会自动调用DSSP计算pose二级结构。
```
<SecondaryStructure name="(&string, required)"
    residue_selector="(&string, TrueSelector)"
    use_abego="(&int, optional)"
    blueprint="(&string)"
    ss="(&string)"
    abego="(&string)"
    compute_pose_secstruct_by_dssp="(&bool, false)"
    threshold="(&real, 1.0)" />
```
1. residue_selector：氨基酸选择器，预先定义
2. ss：提供理想二级结构字符串描述，比如“HHHHHLLLHHHHH”，其长度必须与pose中氨基酸数目保持一致
3. use_dssp：false表示使用pose中储存的二级结构（如果有的话），true表示使用DSSP计算
4. blueprint：用于设定所需pose二级结构的蓝图文件（不要与ss选项混用，Override）
5. abego：所需的abego值，如"XAAAAGBABBBB"，长度必须与pose氨基酸数目一致
6. use_abego：0 or 1. true表示二级结构需要match在abego选项中定义的string描述。
7. threshold：必须至少match二级结构的分数。默认为1，即全部match才会通过该filter
>Secondary Structure specification :
>
>E: sheet
>
>H: helix
>
>h: not helix (so either E or L)
>
>L: loop
>
>D: wildcard (allows anything)

```
使用蓝图文件
<SecondaryStructure name="ss_filter1"  use_abego="1" blueprint="input.blueprint" />

使用ss字段
<FILTERS>
        <SecondaryStructure name="ss" ss="LEEEEEEEELLLLEEEEEEEELLLLHHHHHHHHHHHHHHHLLLEEEEEEELLLHHHHHHHHHHHHHHHHHLLLLhhEEEEELLEEEEEEEL"/>
</FILTERS>
<MOVERS>
         <Dssp name="dssp"/>
</MOVERS>
<PROTOCOLS>
    <Add mover="dssp"/>
    <Add filter="ss"/>
</PROTOCOLS>
```

**SecondaryStructureCount**：基于单个二级结构元件计数的filter。计算给定类型的DSSP定义的二级结构的类型的数目。
```
<SecondaryStructureCount name="(&string)"
    filter_helix_sheet="(1 &bool)" filter_helix="(0 &bool)" filter_sheet="(0 &bool)" filter_loop="(0 &bool)"
    num_helix_sheet="(0 &int)" num_helix="(0 &int)" num_sheet="(0 &int)" num_loop="(0 &int)"
    min_helix_length="(4 &int)" max_helix_length="(9999 &)"
    min_sheet_length="(3 &int)" max_sheet_length="(9999 &)"
    min_loop_length="(1 &int) " max_loop_length="(9999 &)"
    return_total="(false &bool)"
    residue_selector="("" &string)" min_element_resis="(1 &int)" />
```
1. filter_helix, filter_sheet, filter_loop: true 分别表示在helix，sheet，loop二级结构上进行过滤
2. filter_helix_sheet：filter on helix and sheet
3. num_helix，num_sheet，num_loop，num_helix_sheet：需要多少数目的对应二级结构才能通过该filter
4. min_helix_length，max_helix_length：最少和最大的helix氨基酸数目才会被当作一个helix，默认值4和9999
5. min_sheet_length，max_sheet_length，min_loop_length，max_loop_length与上一条类似
6. return_total：true表示将过滤的二级结构元件的总数目记录到score文件。默认为0.
7. residue_selector：氨基酸选择器
8. min_element_resis：一个二级结构原件的最少残基数目（作为计数基础），默认为1.

**SecondaryStructureHasResidue**：计算二级结构元件部分位点是否含有N个或更多的某特定残基。在de novo设计中，用于检查各个二级结构是否存在至少一个疏水残基等用途。
```
<SecondaryStructureHasResidue name="(&string)" secstruct_fraction_threshold="(1.0 &Real)" res_check_task_operations="(&string)" ss_select_task_operations="(&string)" required_restypes="(VILMFYW &string)" nres_required_per_secstruct="(1 &int)" filter_helix="(1 &bool)" filter_sheet="(1 &bool)" filter_loop="(0 &bool)" min_helix_length="(4 &int)" min_sheet_length="(3 &int)" min_loop_length="(1 &int)" />
```
1. secstruct_fraction_threshold:被考虑计算在内的二级结构元件的分数（检查是否存在N个或更多的指定残基）
2. res_check_task_operations：指定哪些残基用于检查正确的残基类型
3. ss_select_task_operations：用于检查的已存在二级结构元件的残基
4. required_restypes：默认为非ALA的疏水类型，使用氨基酸的单字母缩写即可
5. nres_required_per_secstruct：默认仅需一个残基存在在一个二级结构元件，作为pass
6. filter_helix/sheet/loop：yes/no 是否考虑这些二级结构类型

**HelixKink**：基于helix拓扑结构的filter
```
<HelixKink name="(&string)" bend="(20, &Real)" resnums="(&string)" helix_start="(1 &int)"  helix_end="(1 &int)"/>
```
1. bend: helix扭曲角度的cutoff值
2. resnums：逗号分隔的残基列表，任何helix含有该列表中的残基的都会被扫描计算。默认扫描整个蛋白
3. helix_start 和helix_end 可以申明一个连续helix被考虑的起始和终止范围，如此符合要求的helix才会被计算。默认为1（用户需要设置该值！）

**Geometry**：基于键的几何性质和omega角度的constraint的filter
```
<Geometry name="(&string)"
  omega="(165&Real)"
  cart_bonded="(20 &Real)"
  cstfile="('none' &string)"
  cst_cutoff="(10000.0 &Real)"
  start="(1 &residue number)"
  end="(100000 &residue number)"
  residue_selector="( '' &string )"
  count_bad_residues="( false &bool )" />
```
1. omega: 肽键平面的omega角度的cutoff值，Cis-Pro也会被考虑
2. cart_bonded：键角和键长的惩罚分数
3. cstfile：给定constraint 文件，并使用该文件中的限制施加给pose。仅成对原子的constraint会被使用。constraint 打分将会被检查是否低于设定的cst_cutoff值，超过了即fail
3. cst_off: cstfile的附加选项
4. start，end：起始和终止的残基numbering
5. residue_selector：残基选择器，仅在该选择部分进行filter
6. count_bad_residues：true表示报告没有通过该filter的残基数目（report_sm）；false表示filter score为0或1（所有残基都失败/成功）

**HSSTriplet**：评估给定的helix-strand-strand三联（HSS triplet）结构。计算strand pair与helix之间的距离以及sheet平面和helix之间的角度。若距离计算值处于min_dist和max_dist选项设定值且角度计算值处于min_angle和max_angle选项设定值，则返回true。也可以基于output_id和output_type给出数值。
```
<HSSTriplet name="(&string)" hsstriplets="('' &string)" blueprint="('' &string)" min_dist="(7.5 &Real)" max_dist="(13.0 &Real)" min_angle="(-12.5 &Real)" max_angle="(90.0 &Real)" output_id="(1 &bool)" output_type="('dist' &string)" />
```
1. hsstriplets：描述HSS triplet的字符串。该选项或者blueprint 蓝图文件必须有一个被指定，且不能同时被指定。格式为i,j-k，i为整数表示helix的编号，j为strand1的编号（从N端到C端），k是strand2的编号
2. blueprint：二级结构的蓝图文件，需要包含HSS信息

**PreProlineFilter**：基于脯氨酸前一个残基的不合理扭转角的filter
```
<PreProline name="prepro" use_statistical_potential="(&bool)" residue_selector="(&residue selector)" />
```
螺旋空间（Abego，ABEGO type A）在pro之前一般不倾向形成，但rosetta可能捕捉不到该信息。在默认模式下，该filter会检查所有pro残基之前的所有残基，计算那些非B类型和非E类型（越少越好）。
1. use_statistical_potential：true表示使用基于拉式图构象空间bicublic spline fit会被用来评估该扭转角。false表示使用不合理扭转角的bin范围内的残基被计数

**LoopAnalyzerFilter**：使用 LoopAnalyzerMover 计算与loop相关的一些metrics
```
<LoopAnalyzerFilter name="&string" use_tracer="(&bool)" loops_file="(&string)" >
    <Loop start="(&int)" stop="(&int)" cut="(&int)" skip_rate="(0.0 &real)" rebuild="(no &bool)"/>
</LoopAnalyzerFilter>
```
1. 亚标签loop可以进一步详细定义loop
2. loops_file：可以传入文件
3. use_tracer：false表示结果在每次运行结束储存在PoseExtraScores / DataCache中；true表示结果打印到tracer对象中


## 8. Packing/Connectivity相关Filter

**CavityVolume**：用于计算蛋白内部空腔体积，总是返回true。使用will sheffler的packing代码（packstat）计算蛋白内部空腔的体积大小，单位Angstrom^3。20大约为一个碳原子体积大小。注意算法本质具有随机性。
```
<CavityVolume name="(&string)" />

example：
<FILTERS>
    <CavityVolume name="cav_vol" />
</FILTERS>
<PROTOCOLS>
    <Add filter_name="cav_vol" />
</PROTOCOLS>
```


**AverageDegree**：计算与一个残基选择部分的定义距离内的残基数目。能够区分从天然复合物中区分没有相互作用的design。许多没有相互作用的design都存在没有很好嵌合的表面。
```
<AverageDegree name="(&string)" threshold="(0&Real)" distance_threshold="(&10.0)" task_operations="(comma-delimited list)"/>
```
1. threshold：至少需要存在多少个氨基酸在选择部分氨基酸的范围内
2. distance_threshold：定义计算距离的范围大小
3. task_operations定义针对哪些残基进行该计算


**PackStat**：基于packing统计量的filter
```
<PackStat name="(&string)" threshold="(0.58 &Real)" chain="(0 &integer)" repeats="(1 &integer)"/>
```
1. threshold：最低阈值
2. chain：在计算packstate之前从哪一个jump开始分离复合物。0表示不分离
3. repeats：重复计算次数


**Holes**：寻找packing中的空腔。仍然是使用will sheffler的packing代码（packstat）计算蛋白内部空腔的体积大小。但是这一允许传入氨基酸选择器，仅使用部分氨基酸（蛋白部分进行计算）。但是值得注意是，这里的计算仍然是把pose作为整体进行计算，只是report得分的时候，只有在选择器中的原子才会被计算总和。（Holes打分是对单个原子/残基进行计算得分值得总和。）结果如果正说明比之native结构更多空腔，负说明更少空腔。
```
<Holes name="(&string)" threshold=“(&real)" residue_selector="(&string)" normalize_per_atom="(&bool)" normalize_per_residue="(&bool)" exclude_bb_atoms="(&bool)"  />
```
1. threshold：最高阈值。高于该值，将不通过。
2. exclude_bb_atoms：不考虑骨架原子的holes贡献
3. normalize_per_residue：对于选中的残基部分进行归一化（不推荐使用，默认为false）
4. normalize_per_atom：同上（对原子）

**InterfaceHoles**：在蛋白-蛋白接触界面上计算空腔，使用Will Sheffler's packstat的脚本。报告的打分是指bound和unbound构象的holes得分的差值。需要开启-holes:dalphaball选项。
```
<InterfaceHoles name="(&string)" jump="(1 &integer)" threshold="(200 &integer)"/>
```
1. jump：计算InterfaceHoles的jump
2. threshold：最高阈值

**NeighborType**：在一个残基附近搜索是否存在一种设定的残基类型的filter。
```
<NeighborType name="(neighbor_filter &string)" res_num/pdb_num="(&string)" distance="(8.0 &Real)">
        <Neighbor type="(&3-letter aa code)"/>
</NeighborType>
```

**ResInInterface**：基于界面上残基总数的filter。
```
<ResInInterface name="(riif &string)" residues="(20 &integer)" jump_number="(1 &integer)"/>
```
1. 需要制定jump说明计算哪一个接触界面
2. threshold为最低阈值，超过设定值，返回true。

**ShapeComplementarity**：基于界面形状互补的filter。计算 Lawrence & Coleman形状互补系数。
```
<ShapeComplementarity name="(&string)" min_sc="(0.5 &Real)" min_interface="(0 &Real)" verbose="(0 &bool)" quick="(0 &bool)" jump="(1 &int)" sym_dof_name="('' &string)" residues1="(comma-separated list)" residues2="(comma-separated list)" residue_selector1="('' &string)" residue_selector2="('' &string)" write_int_area="(1 &bool)" write_median_dist="(0 &bool)" max_median_dist="(1000 &Real)"/>
```
1. min_sc：最低形状互补系数阈值
2. quick：使用快速但降低精确度的算法
3. sym_dof_name：对于具有对称性的pose，计算哪一个dof的interface
4. residues1 & residues2 ：显式说明哪些残基是在interface的哪一边
5. residue_selector1 & residue_selector2：显式，同上（残基选择部分）
6. write_int_area：将得分写入score 文件
7. write_median_dist：写入median distance
8. max_median_dist：median distance最高阈值

**SSShapeComplementarity**：基于二级结构形状互补系数的filter
```
<SSShapeComplementarity name="(&string;)" blueprint="(&string;)"
        secstruct="(&string;)" residue_selector="(&string;)"
        verbose="(false &bool;)" loops="(true &bool;)" helices="(true &bool;)"
        min_sc="(0.0 &real;)" confidence="(1.0 &real;)" />
```
使用与上一条filter相同的脚本，计算基于二级结构的Lawrence & Coleman形状互补系数。二级结构元件（loop，helix等）从蛋白整体切下，然后计算其与剩余蛋白部分的互补情况。filter被调用的时候，蛋白还原，而后使用下一个二级结构元件进行相同计算。返回值为sum( sc_i × a_i )/a_tot，sc_i为第i个二级结构元件的形状互补系数值，a_i为第i个二级结构元件与剩余部分的相互作用面积，a_tot为所有二级结构元件的蛋白内面积的总和。目前不支持sheet结构。可以使用blueprint指定二级结构。若总体的形状互补系数低于min_sc值，返回false，否则返回true。

**SpecificResiduesNearInterface**：基于界面附近特定残基的filter。即要求pose的界面附近需要含有指定残基。
```
<SpecificResiduesNearInterface name="(&string)" task_operation="(&string)"/>
```
例如taskop中design的残基需要靠近interface。

## 9. Burial相关filter

**TotalSasa**：基于pose总溶剂可及表面积的filter。高于设定阈值（threshold），返回true。
```
<TotalSasa name="(sasa_filter &string)" threshold="(800 &float)" upper_threshold="(1000000000000000 &float)" hydrophobic="(0&bool)" polar="(0&bool)" task_operations="(comma-delimited list of operations &string)" />
```
1. upper_threshold：最大溶剂可及表面积
2. hydrophobic：仅计算与疏水性残基相关的可及表面积
3. polar：仅计算与极性残基相关的可及表面积
4. task_operations：进报告可以被pack的残基部分（有taskop指定）的SASA值。若没有指定，则会计算所有残基的SASA
5. report_per_residue_sasa：报告单个残基的SASA


**Sasa**：基于界面上溶剂可及表面积的filter。高于设定阈值，返回true。
```
<Sasa name="(sasa_filter &string)" threshold="(800 &float)" upper_threshold="(1000000000000000 &float)" hydrophobic="(0&bool)" polar="(0&bool)" jump="(1 &integer)" sym_dof_names("" &string)/>
```
1. upper_threshold：最大溶剂可及表面积
2. jump：用于计算SASA的界面jump
3. sym_dof_names：对于存在对称定义的pose，进一步指定计算何界面的SASA

**ResidueBurial**：在目标残基界面的相互作用距离范围内的残基数目的filter。简单来说就是计算目标残基的某相互作用距离（distance选项）下范围的其他残基数。若设定neighbors为1，既仅仅检查界面附近是否存在残基。
```
<ResidueBurial name="(&string)" res_num/pdb_num="(&string)" distance="(8.0 &Real)" neighbors="(1 &Integer)" task_operations="(&comma-delimited list of taskoperations)" residue_fraction_buried="(0.0001 &Real)"/>
```
1. task_operations： taskop指定的可设计的残基。任意一个残基通过burial threshold则返回true
2. residue_fraction_buried：被taskop定义为designable的总残基的分数，默认为0.0001

**BuriedSurfaceArea**：计算一个pose或selection的包埋的表面积。若低于设定阈值，返回false。该filter仅适用在L型或D型天然氨基酸，其他类型一律为0。
```
<BuriedSurfaceArea name=(string)
     select_only_FAMILYVW=(bool,"false") filter_out_low=(bool,"true")
     cutoff_buried_surface_area=(real,"500") atom_mode=(string, "all_atoms")
     residue_selector=(string) confidence=(real,"1.0")
/>
```
1. select_only_FAMILYVW： true表示仅计算FAMILYVW这些残基；false表示所有的残基都会被计算。会与residue_selector进行取交集进行计算。
2. filter_out_low ：默认为true，表示pose/selection计算值低于阈值将被reject掉
3. cutoff_buried_surface_area ：默认为500
4. atom_mode：默认为“all_atoms”，也可以是"hydrophobic_atoms"、"polar_atoms"

**ExposedHydrophobics**：对每个疏水残基进行的SASA计算(A, F, I, M, L, W, V, Y)。score返回溶剂暴露的疏水残基数目和暴露程度。对于每一个疏水残基，SAS高于设定cutoff值（默认20），则将 SASA - sasa_cutoff的计算值加和到score上。若最终score低于定义的threshold，返回true。
```
<ExposedHydrophobics name="(&string)" sasa_cutoff="(20 &Real)" threshold="(-1 &Real)" />
```

## 10. Comparison相关Filter

**RelativePose**：比较pose相比于另一个不同pose结构的filter计算值。
```
<RelativePose name="(&string)" pdb_name="(&string)" filter="(&string)" relax_mover="(null &string)" dump_pose="('' &string)" alignment="(&string; see below)" scorefxn="(score12 &string)" packing_shell="(8.0 &Real)" thread="(1 &bool)" baseline="(1 &bool)" unbound="(0 &bool)" copy_stretch="(1&bool)" rtmin="(0 &bool)" symmetry_definition="('' &string)" copy_comments="('' &comma-delimited list)">
```
1. pdb_name：参照pose读入的pdb文件
2. filter：需要计算pose的何种filter值
3. dump_pose：是否输出pose
4. alignment：需要在disk读入的pose和当前pose的哪一个segment进行align，默认为1 -> nres，例如也可以指定为3A:1B,4A:2B,5A:6B，意思为align disk读入的pose的3A-5A到1B，2B和6B位置。
5. packing_shell：每个残基周围多大范围内进行repack

**Rmsd**：计算当前pose与参照pose的rmsd值的filter。
```
<Rmsd name="(&string;)" reference_name="(&string;)" symmetry="(0 &bool;)"
        chains="(&string;)" superimpose_on_all="(false &bool;)"
        superimpose="(1 &bool;)" threshold="(5 &real;)" by_aln="(&bool;)"
        aln_files="(&string;)" template_names="(&string;)"
        query_names="(&string;)" rms_residues_from_pose_cache="(1 &string;)"
        confidence="(1.0 &real;)" >
    <rmsd resnum="(&refpose_enabled_residue_number;)"
            pdb_num="(&refpose_enabled_residue_number;)" />
    <span begin_res_num="(&refpose_enabled_residue_number;)"
            begin_pdb_num="(&refpose_enabled_residue_number;)"
            end_res_num="(&refpose_enabled_residue_number;)"
            end_pdb_num="(&refpose_enabled_residue_number;)" />
    <span_two begin_native="(&non_negative_integer;)"
            end_native="(&non_negative_integer;)"
            begin_pose="(&non_negative_integer;)"
            end_pose="(&non_negative_integer;)" />
</Rmsd>
```
1. reference_name：参照pose的名字（使用SavePoseMover 可以创建参照pose）
2. superimpose_on_all：是否叠合所有残基
3. threshold：最高阈值
4. by_aln：依据alignment文件进行rmsd计算，align_files必须指定
5. template_names：逗号分隔的template的名字
6. rms_residues_from_pose_cache：是否pose的缓存中存在RMS残基

**SidechainRmsd**：计算单个残基的全原子RMSD （可以包含或不包含骨架原子）。计算rmsd之前不会进行叠合。
```
SidechainRmsd name="(&string)" res1_(res/pdb)_num="(&string)" res2_(res/pdb)_num="(&string)" reference_name="(&string)" include_backbone="(0 &bool)" threshold="(1.0 &real)" />
```
1. threshold：最高阈值

**IRmsd**：计算interface上的RMSD，包含interface上的所有骨架原子。interface残基包括在两边界面8埃范围内的所有残基。
```
<IRmsd name="(&string)" jump="(&int 1)" threshold="(&Real 5)" reference_pose="(&string)" scorefxn="(&string talaris2013)" />
```
1. threshold：最高阈值。比之低的pose会通过该filter
2. reference_pose：需要制定参照pose

**RmsdFromResidueSelectorFilter**：与Rmsd filter类似，只不过对参照pose和当前计算pose分别提供ResidueSelectors。
```
<RmsdFromResidueSelectorFilter name="(&string;)" CA_only="(1 &bool;)"
        use_gdt="(0 &bool;)" superimpose="(1 &bool;)"
        count_residues="(1 &bool;)" threshold="(5.000000 &real;)"
        reference_name="(&string;)" reference_selector="(&string;)"
        query_selector="(&string;)" confidence="(1.0 &real;)" />
```

**SequenceRecovery**：对比参照pose，当前pose有多大的序列回复率的filter。用户通过task_operations提供可design的残基范围。command line可输入-in:file:native 将用户输入的pose文件作为参照，否则将使用起始pose作为参照。
```
<SequenceRecovery name="(&string)" rate_threshold="(0.0 &Real)" task_operations="(comma-delimited list of operations &string)" mutation_threshold="(100 &Size)" report_mutations="(0 &bool)" scorefxn="('talaris2013' &string)" verbose="(0 &bool)" />
```
1. rate_threshold：最低通过阈值（至少需要到多大的回复率）
2. mutation_threshold：突变的最大个数
3. report_mutations：默认false，不会report。true表示不再使用rate作为filter，而使用突变个数。

## 11. bonding相关Filter

**ChainBreak**：基于pose中链断裂数目的filter。这里的break指的是当某键长偏离平均键长（1.33）+/- tolerance (默认0.13为tolerance)
```
< ChainBreak name="(&string)" 
    chain_num="(1 &int)" 
    tolerance="(0.13 &float)"
    threshold="(1 &int)" 
/>
```

**HbondsToResidue**：基于某残基的氢键连接数目的filter。计算与某残基形成氢键的残基数目，且每一个氢键必须要超过一定的energy_cutoff值。对于骨架间的氢键，需要开放bb_bb选项。
```
<HbondsToResidue name="(hbonds_filter &string)" scorefxn="(&string)" partners="(&integer)" energy_cutoff="(-0.5 &float)" backbone="(false &bool)" bb_bb="(false &bool)" sidechain="(true &bool)" residue="(&string)" from_other_chains="(true &bool)" from_same_chain="(true &bool)" residue_selector="(&string)">
```
1. partners：预期的氢键partner个数，最低阈值。至少需要规定数目的partner，该filter才能通过
2. backbone：是否包含骨架氢键
3. sidechain：是否包含侧链氢键
4. bb_bb：是否记录骨架-骨架氢键
5. from_other_chains：true表示计算与另外的链的氢键partner（default为true）
6. from_same_chain：true表示需要计算本条链内部的partner（default为true）

**SimpleHbondsToAtom**：基于某原子的氢键数目的filter。检查目标原子是否存在至少n_partners指定的氢键partner
```
<SimpleHbondsToAtomFilter name="(hbonds_filter &string)" n_partners="(&integer)" hb_e_cutoff="(-0.5 &float)"
            target_atom_name="(&string)" res_num/pdb_num="(&string)" scorefxn="(&string)"/>
```
1. hb_e_cutoff：氢键能量cutoff值，默认-0.5。只要达标的氢键才会被该filter算作partner。

**HbondsToAtom**：与SimpleHbondsToAtom相似，但逻辑稍复杂。结合了部分HbondsToResidue的选项。
```
<HbondsToAtom name="(hbonds_filter &string)" partners="(&integer)" energy_cutoff="(-0.5 &float)" backbone="(0 &bool)" bb_bb="(0 &bool)" sidechain="(1 &bool)" atomname="(&string)" res_num/pdb_num="(&string)">
```

**PeptideInternalHbondsFilter**：在一个pose或selection中的氢键数目，排除了在threshold范围内的共价连接残基之间的氢键部分。低于设定阈值的pose会被reject掉。在寻找长距离的骨架氢键作用时候比较有用。
```
<PeptideInternalHbondsFilter name="(&string;)" backbone_backbone="(true &bool;)"
        backbone_sidechain="(false &bool;)" sidechain_sidechain="(false &bool;)"
        exclusion_distance="(1 &non_negative_integer;)"
        hbond_energy_cutoff="(-0.25 &real;)" scorefxn="(&string;)"
        residue_selector="(&string;)" hbond_cutoff="(1 &non_negative_integer;)"
        confidence="(1.0 &real;)" />
```
1. backbone_backbone：默认为true，计算骨架-骨架间氢键
2. backbone_sidechain：默认为true
3. sidechain_sidechain：默认为true
4. exclusion_distance：既在该设定范围内共价连接的残基间氢键不必计算
5. hbond_cutoff：最低阈值，默认为1，最少需要的氢键数目




**BuriedUnsatHbonds**：基于被包埋的不饱和氢键的最大数目的filter。
```
# old filter, recapitulates results of this filter before Dec 16 2017
<BuriedUnsatHbonds use_legacy_options="true" name="(&string)" scorefxn="(&string)" jump_number="(1 &Size)" cutoff="(20 &Size)" task_operations="(&string)"/>

# report number of backbone (bb) heavy-atom buns, using residue selector:
#  many native and de novo backbones have at least a couple of legit buried unsats in the backbone; most occur in loops or other flexible regions, but setting this to 0 will throw out potentially good designs; best to set cutoff to a few, or use confidence="0"
<BuriedUnsatHbonds name="new_buns_bb_heavy" residue_selector="(&string)" report_bb_heavy_atom_unsats="true" scorefxn="(&string)" cutoff="4" residue_surface_cutoff="20.0" ignore_surface_res="true" print_out_info_to_pdb="true" />

# report the number of sidechain (sc) heavy-atom buns, using a residue selector
#  if doing design, and residue selector is design space, you want this to be 0
<BuriedUnsatHbonds name="new_buns_sc_heavy" residue_selector="(&string)" report_sc_heavy_atom_unsats="true" scorefxn="(&string)" cutoff="0" residue_surface_cutoff="20.0" ignore_surface_res="true" print_out_info_to_pdb="true" />
```
1. 该filter被更新过，略显复杂。如果需要使用旧版filter，指定use_legacy_options为true即可

**BuriedUnsatHbonds2**：与上一个相似，只是算法不同。(具体如何不同，官方也没有明确给出)
```
<BuriedUnsatHbonds2 name="(&string;)" jump_number="(1 &non_negative_integer;)"
        cutoff="(20 &non_negative_integer;)" generous_hbonds="(true &bool;)"
        sasa_burial_cutoff="(0.01 &real;)" AHD_cutoff="(120 &real;)"
        dist_cutoff="(3.0 &real;)" hxl_dist_cutoff="(3.5 &real;)"
        sulph_dist_cutoff="(3.3 &real;)" metal_dist_cutoff="(2.7 &real;)"
        scorefxn="(&string;)"
        task_operations="(&task_operation_comma_separated_list;)"
        packer_palette="(&named_packer_palette;)" confidence="(1.0 &real;)" />
```
1. threshold：最高阈值
2. sasa_burial_cutoff：定义残基被包埋与否的sasa阈值
3. AHD_cutoff：氢键Acceptor - H - Donor之间角度阈值
4. dist_cutoff：氢键距离阈值
5. hxl_dist_cutoff：氢键距离羟基的距离阈值
6. sulph_dist_cutoff：氢键距离硫原子距离阈值
7. metal_dist_cutoff：氢键距离金属例子的阈值

**OversaturatedHbondAcceptorFilter**：过饱和氢键受体的filter。超过一个供体的氢键受体属于此类。
```
<OversaturatedHbondAcceptorFilter name="(&string)" scorefxn="(&string)" max_allowed_oversaturated="(0 &int)" hbond_energy_cutoff="(-0.1 &Real)" consider_mainchain_only="(true &bool)" donor_selector="(&string)" acceptor_selector="(&string)" >
```
选项含义大多不言自明。
1. max_allowed_oversaturated：最大允许过饱和数，默认为0。既通过的pose不含有任何过饱和氢键受体。

**DisulfideFilter**：二硫键的filter。计算界面上二硫键是否存在。
```
<DisulfideFilter name="&string" targets="(&string)"/>
```
1. targets: 逗号分隔的残基numbering的列表，兼容rosetta numbering和pdb numbering。列表中残基需要处于界面上

**AveragePathLength**：计算蛋白质“图网络表示”的最短途径距离，图表示中残基为节点，共价键为边。
```
<AveragePathLength name="&string" path_tightness="(1 &Real)" max_path_length="(10000 &Real)"/>
```
原也是设计用来搜索二硫键相关metric（不如DisulfideEntropyFilter准确）。在设定最高阈值内，pose可以通过该filter。

threshold = (0.1429 n) + 0.8635 + path_tightness

n为链中残基数目，path_tightness需要用户定义。

**DisulfideEntropy**：由于二硫键形成构象熵值改变的计算。
```
<DisulfideEntropy name="&string" tightness="(0 &Real)" lower_bound="(0 &Real)"/>
```
计算由于二硫键形成导致deltaSconf_folding的变化。S_conf仅表示蛋白链的构象熵：deltaS_conf_folding = S_conf_folded - S_conf_unfolded。

deltaG_folding = deltaH_folding - T (deltaS_conf_folding + deltaS_other)

若deltaS_conf_folding计算为正，说明folded状态有更大的稳定性。

threshold = (0.1604 * residues) + (1.7245 * disulfides) + 5.1477 + tightness

其中仅tightness需要用户定义，用以调整阈值。设定为1，大约82%天然蛋白可以通过该filter，设定为-1，仅19%能够通过。